In [ ]:
from typing import Tuple

%run imputers/interpolate.ipynb
%run imputers/median.ipynb
%run imputers/mode.ipynb
%run imputers/mean.ipynb
%run imputers/knn.ipynb
%run imputers/missforest.ipynb
%run imputers/hotdeck.ipynb
%run imputers/regression.ipynb
%run imputers/softimpute.ipynb

imputers = {
    "interpolation": {
        "title": "Interpolation",
        "function": interpolate
    },
    "median imputation": {
        "title": "Median imputation",
        "function": median_imputation
    },
    "mode imputation": {
        "title": "Mode imputation",
        "function": mode_imputation
    },
    "mean imputation": {
        "title": "Mean imputation",
        "function": mean_imputation
    },
    "KNN imputation": {
        "title": "K-Nearest-Neighbors imputation",
        "function": KNNImputation
    },
    "MissForest imputation": {
        "title": "MissForest imputation",
        "function": MissForestImputation
    },
    "Softimpute": {
        "title": "Softimpute",
        "function": SoftImputation
    },
    "Hotdeck imputation": {
        "title": "Hot-deck imputation",
        "prehook": hotdeck_prehook,
        "function": hotdeck
    },
    "stochastic regression": {
        "title": "Stochastic regression",
        "function": lambda df, _: stochastic_regression(df, dfloader.targets)
    },
    "deterministic regression": {
        "title": "Deterministic regression",
        "function": lambda df, _: deterministic_regression(df, dfloader.targets)
    },

}

imputer_select = widgets.Dropdown(
    options=list(imputers.keys()),
    value=None,
    description='Imputer:'
)
display(imputer_select)
imputer = None
imputation_date = None
imputed_dfs: [pd.DataFrame] = None
imputer_configs = []

imputation_status = widgets.HTML(value="")
display(imputation_status)


def do_imputation():
    global imputed_dfs, imputer, imputation_date
    imputation_date = datetime.now().replace(second=0, microsecond=0)
    imputed_dfs = []
    for i in range(len(dfs_with_gaps)):
        imputation_status.value = f"Running imputation... ({i}/{len(dfs_with_gaps)})"
        dataset_config['current_gap_indices'] = gaps_indices[i]
        result, config = imputer["function"](dfs_with_gaps[i].copy(), dataset_config)
        imputer_configs.append(config)
        imputed_dfs.append(result)
    imputation_status.value = "Imputation complete"


def on_imputer_select_change(evt):
    global imputer
    imputer = imputers[evt.new]
    if "prehook" in imputer.keys():
        imputer["prehook"](lambda: do_imputation())
    else:
        do_imputation()


imputer_select.observe(on_imputer_select_change, 'value')
